In [6]:
# Cell 1: Import Libraries
import pandas as pd
import pyodbc
import numpy as np
from datetime import datetime
import os

print("📦 Libraries loaded")

from config import (
    MODELS_DIR, DATA_DIR, LAG_DAYS, ROLLING_WINDOWS, 
    CHUNK_SIZE, DEFAULT_MODEL_PARAMS,
    SQL_SERVER, SQL_DATABASE, SQL_TRUSTED_CONNECTION,
    SQL_USERNAME, SQL_PASSWORD,
    FEATURE_GROUPS, PRODUCTIVITY_FEATURES, DATE_FEATURES, ESSENTIAL_LAGS, ESSENTIAL_WINDOWS,
    enterprise_logger
)


# Cell 2: Database Connection
SQL_SERVER = SQL_SERVER
SQL_DATABASE = SQL_DATABASE

conn_str = f'DRIVER={{SQL Server}};SERVER={SQL_SERVER};DATABASE={SQL_DATABASE};Trusted_Connection=yes;'
print(f"🔗 Connecting to: {SQL_SERVER}/{SQL_DATABASE}")

# Cell 3: Extract Data (Only Safe Features)
query = """
SELECT Date, PunchCode as WorkType, Hours, NoOfMan, SystemHours, NoRows as Quantity, SystemKPI 
FROM WorkUtilizationData 
WHERE PunchCode IN (206, 213) 
AND Hours > 0 
AND NoOfMan > 0 
AND SystemHours > 0 
AND NoRows > 0
AND Date < '2025-05-01'
ORDER BY Date
"""

try:
    conn = pyodbc.connect(conn_str)
    df = pd.read_sql(query, conn)
    conn.close()
    
    print(f"✅ Data extracted: {df.shape}")
    print(f"📅 Period: {df['Date'].min()} to {df['Date'].max()}")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")

# Cell 4: Clean Data & Handle Decimals
df['Date'] = pd.to_datetime(df['Date'])

# Handle decimals appropriately
df['NoOfMan'] = df['NoOfMan'].round(0).astype(int)
df['SystemHours'] = df['SystemHours'].round(1)
df['SystemKPI'] = df['SystemKPI'].round(2)
df['Hours'] = df['Hours'].round(1)
df['Quantity'] = df['Quantity'].round(0).astype(int)

print("🔧 Decimals handled")

# Cell 5: Create Meaningful Features (Optimized)
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['Month'] = df['Date'].dt.month
df['WeekNo'] = df['Date'].dt.isocalendar().week
df['Year'] = df['Date'].dt.year

# Simplified feature creation - focus on schedule differences
df['ScheduleType'] = np.where(df['WorkType'] == 206, '6DAY', '5DAY')
df['CanWorkSunday'] = np.where(df['WorkType'] == 206, 1, 0)
df['IsSunday'] = (df['DayOfWeek'] == 6).astype(int)
df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)

print("📅 Optimized features created")

# Cell 6: Data Quality Check
print("\n📋 DATA SUMMARY")
print("=" * 40)
print(f"Total records: {len(df):,}")
print(f"Date range: {df['Date'].min().date()} to {df['Date'].max().date()}")

print(f"\nPunch Code distribution:")
print(df['WorkType'].value_counts())

print(f"\nSchedule types:")
print(df['ScheduleType'].value_counts())

print(f"\nHours statistics:")
print(df.groupby('WorkType')['Hours'].describe())

# Cell 7: Save to Excel
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'hours_prediction_data_{timestamp}.xlsx'

with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    # Main dataset for model training
    df.to_excel(writer, sheet_name='ModelData', index=False)
    
    # Summary statistics
    summary = df.groupby('WorkType').agg({
        'Hours': ['count', 'mean', 'std', 'min', 'max'],
        'Quantity': ['mean', 'std'],
        'SystemHours': ['mean', 'std'],
        'NoOfMan': ['mean', 'std']
    }).round(2)
    summary.to_excel(writer, sheet_name='Summary')

print(f"✅ Excel saved: {filename}")

# Cell 8: Model Readiness Check (Fixed Version)
print(f"\n🎯 MODEL READINESS")
print("=" * 30)

# Check sufficient data per punch code
for code in [206, 213]:  # Changed to integers to match your data
    code_data = df[df['WorkType'] == code]
    
    print(f"\nPunchCode {code}:")
    print(f"  Total records: {len(code_data)}")
    
    # Only try to access schedule type if records exist
    if len(code_data) > 0:
        print(f"  Schedule: {code_data['ScheduleType'].iloc[0]}")
        print(f"  Avg hours: {code_data['Hours'].mean():.1f}")
    else:
        print(f"  ⚠️ No records found for this punch code")

# Feature correlation with target (only if we have data)
if len(df) > 0:
    numeric_features = ['Quantity', 'SystemHours', 'SystemKPI', 'NoOfMan', 'DayOfWeek', 'Hours']
    corr_with_hours = df[numeric_features].corr()['Hours'].sort_values(ascending=False)

    print(f"\n📈 Feature correlation with Hours:")
    for feature, corr in corr_with_hours.items():
        if feature != 'Hours':
            print(f"  {feature}: {corr:.3f}")
else:
    print("\n⚠️ No data available for correlation analysis")

print(f"\n✅ Dataset ready for Hours prediction model!")
print(f"📁 File: {filename}")
print(f"📊 Features: {len(df.columns)} columns, {len(df)} rows")

# Display final feature list only if we have data
if len(df) > 0:
    print(f"\n📋 Features available for model:")
    feature_cols = [col for col in df.columns if col not in ['Date']]
    for i, col in enumerate(feature_cols, 1):
        print(f"  {i:2d}. {col}")
else:
    print("\n⚠️ No features available - empty dataset")

📦 Libraries loaded
🔗 Connecting to: 192.168.1.43/ABC


C:\Users\amila.g\AppData\Local\Temp\ipykernel_8812\2463885388.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ Data extracted: (3263, 7)
📅 Period: 2019-07-01 to 2025-04-30
🔧 Decimals handled
📅 Optimized features created

📋 DATA SUMMARY
Total records: 3,263
Date range: 2019-07-01 to 2025-04-30

Punch Code distribution:
WorkType
206    1795
213    1468
Name: count, dtype: int64

Schedule types:
ScheduleType
6DAY    1795
5DAY    1468
Name: count, dtype: int64

Hours statistics:
           count        mean        std  min    25%    50%      75%    max
WorkType                                                                  
206       1795.0  168.635265  69.030560  5.1  121.8  174.7  210.600  401.9
213       1468.0   24.997616  14.840798  1.2   14.3   21.5   32.825  102.1
✅ Excel saved: hours_prediction_data_20250610_151147.xlsx

🎯 MODEL READINESS

PunchCode 206:
  Total records: 1795
  Schedule: 6DAY
  Avg hours: 168.6

PunchCode 213:
  Total records: 1468
  Schedule: 5DAY
  Avg hours: 25.0

📈 Feature correlation with Hours:
  NoOfMan: 1.000
  SystemHours: 0.950
  Quantity: 0.594
  DayOfWeek: -

In [7]:
# Remove NoOfMan (since it's Hours/8 - data leakage)
df = df.drop(columns=['NoOfMan'])

# Display cleaned dataset preview
print("\n🔍 FINAL DATASET FOR MODELING")
print("=" * 50)
print(f"Shape: {df.shape} (Rows, Columns)")
print("\n📋 First 5 Rows:")
display(df.head())

print("\n📊 Summary Statistics:")
display(df.describe().round(2))

print("\n📅 Temporal Coverage:")
print(f"Start Date: {df['Date'].min()}")
print(f"End Date: {df['Date'].max()}")

print("\n🔢 Feature Correlation with Hours:")
corr = df.select_dtypes(include=np.number).corr()['Hours'].sort_values(ascending=False)
display(corr[1:])  # Exclude self-correlation


🔍 FINAL DATASET FOR MODELING
Shape: (3263, 14) (Rows, Columns)

📋 First 5 Rows:


,Date,WorkType,Hours,SystemHours,Quantity,SystemKPI,DayOfWeek,Month,WeekNo,Year,ScheduleType,CanWorkSunday,IsSunday,IsWeekend
0,2019-07-01,206,251.9,108.2,1049,9.69,0,7,27,2019,6DAY,1,0,0
1,2019-07-01,213,30.0,0.1,15,128.57,0,7,27,2019,5DAY,0,0,0
2,2019-07-02,213,60.6,0.1,20,167.44,1,7,27,2019,5DAY,0,0,0
3,2019-07-02,206,243.9,136.7,1095,8.01,1,7,27,2019,6DAY,1,0,0
4,2019-07-03,213,45.3,0.1,17,298.54,2,7,27,2019,5DAY,0,0,0



📊 Summary Statistics:


,Date,WorkType,Hours,SystemHours,Quantity,SystemKPI,DayOfWeek,Month,WeekNo,Year,CanWorkSunday,IsSunday,IsWeekend
count,3263,3263.00,3263.00,3263.00,3263.00,3263.00,3263.00,3263.00,3263.0,3263.00,3263.00,3263.00,3263.00
mean,2022-05-29 06:01:26.055776768,209.15,104.01,50.15,786.55,72.22,2.35,6.55,26.74,2021.90,0.55,0.09,0.09
min,2019-07-01 00:00:00,206.00,1.20,0.00,2.00,3.59,0.00,1.00,1.0,2019.00,0.00,0.00,0.00
25%,2020-12-10 00:00:00,206.00,23.35,4.40,416.00,10.46,1.00,3.00,13.0,2020.00,0.00,0.00,0.00
50%,2022-05-31 00:00:00,206.00,64.80,41.00,766.00,13.63,2.00,7.00,28.0,2022.00,1.00,0.00,0.00
75%,2023-11-11 00:00:00,213.00,182.20,94.25,1101.50,116.22,4.00,10.00,40.0,2023.00,1.00,0.00,0.00
max,2025-04-30 00:00:00,213.00,401.90,194.00,4944.00,3601.44,6.00,12.00,53.0,2025.00,1.00,1.00,1.00
std,NaN,3.48,88.47,45.89,470.76,113.07,1.77,3.47,15.12,1.73,0.50,0.28,0.29



📅 Temporal Coverage:
Start Date: 2019-07-01 00:00:00
End Date: 2025-04-30 00:00:00

🔢 Feature Correlation with Hours:


SystemHours      0.949664
CanWorkSunday    0.807796
Quantity         0.594010
WeekNo           0.143073
Month            0.140885
Year            -0.052517
IsSunday        -0.122695
IsWeekend       -0.132040
DayOfWeek       -0.194834
SystemKPI       -0.503317
WorkType        -0.807796
Name: Hours, dtype: float64

In [ ]:
import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import joblib  # For model saving

# 1. Load Preprocessed Data (after removing NoOfMan)
print("🔁 Loading cleaned dataset...")
df = pd.read_excel("hours_prediction_data_20250610_151147.xlsx") 

# 2. Split by WorkType
print("\n🔀 Splitting by WorkType...")
df_206 = df[df['WorkType'] == 206].copy()
df_213 = df[df['WorkType'] == 213].copy()

# 3. Feature Engineering for Each WorkType
def engineer_features(df):
    """Create temporal features from Date column"""
    df['DayOfWeek'] = df['Date'].dt.weekday  # Monday=0, Sunday=6
    df['IsSunday'] = (df['DayOfWeek'] == 6).astype(int)
    df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)
    return df

df_206 = engineer_features(df_206)
df_213 = engineer_features(df_213)

# 4. Define Features/Target
features_206 = ['SystemHours', 'Quantity', 'IsSunday', 'DayOfWeek']
features_213 = ['SystemHours', 'Quantity', 'IsWeekend']

X_206, y_206 = df_206[features_206], df_206['Hours']
X_213, y_213 = df_213[features_213], df_213['Hours']

# 5. Train Models
print("\n🤖 Training models...")
model_206 = RandomForestRegressor(n_estimators=100, random_state=42)
model_213 = RandomForestRegressor(n_estimators=100, random_state=42)

model_206.fit(X_206, y_206)
model_213.fit(X_213, y_213)

# 6. Save Models
joblib.dump(model_206, 'model_206.joblib')
joblib.dump(model_213, 'model_213.joblib')

# 7. Prediction Function
def predict_hours(worktype: int, date: str, system_hours: float, quantity: int):
    """Predict hours for a future date"""
    # Load appropriate model
    model = joblib.load(f'model_{worktype}.joblib')
    
    # Parse date and calculate features
    dt = datetime.strptime(date, "%Y-%m-%d")
    day_of_week = dt.weekday()
    
    # Create input DataFrame
    if worktype == 206:
        input_data = pd.DataFrame({
            'SystemHours': [system_hours],
            'Quantity': [quantity],
            'IsSunday': [1 if day_of_week == 6 else 0],
            'DayOfWeek': [day_of_week]
        })
    else:
        input_data = pd.DataFrame({
            'SystemHours': [system_hours],
            'Quantity': [quantity],
            'IsWeekend': [1 if day_of_week >= 5 else 0]
        })
    
    # Predict and return
    return model.predict(input_data)[0]

# 8. Example Usage
print("\n🔮 Example Predictions:")
print(f"206 on 2025-05-15: {predict_hours(206, '2025-05-15', 120.5, 800):.1f} hours")
print(f"213 on 2025-05-17: {predict_hours(213, '2025-05-17', 25.0, 50):.1f} hours")

# 9. Model Evaluation (Optional)
print("\n📊 Model Performance:")
X_train_206, X_test_206, y_train_206, y_test_206 = train_test_split(X_206, y_206, test_size=0.2)
model_206.fit(X_train_206, y_train_206)
print(f"206 MAE: {mean_absolute_error(y_test_206, model_206.predict(X_test_206)):.1f} hours")

X_train_213, X_test_213, y_train_213, y_test_213 = train_test_split(X_213, y_213, test_size=0.2)
model_213.fit(X_train_213, y_train_213)
print(f"213 MAE: {mean_absolute_error(y_test_213, model_213.predict(X_test_213)):.1f} hours")

🔁 Loading cleaned dataset...

🔀 Splitting by WorkType...

🤖 Training models...

🔮 Example Predictions:
206 on 2025-05-15: 198.7 hours
213 on 2025-05-17: 38.7 hours

📊 Model Performance:
206 MAE: 20.9 hours
213 MAE: 10.9 hours
